In [4]:
!nvidia-smi

Sat Jul 19 19:53:41 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.230.02             Driver Version: 535.230.02   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A40                     On  | 00000000:19:00.0 Off |                    0 |
|  0%   28C    P8              21W / 300W |      0MiB / 46068MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [5]:
import pandas as pd
import ast
from collections import Counter
import matplotlib.pyplot as plt
import numpy as np
import ast
import csv
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [6]:
DATASET_NAMES = [
                "emotion", 
                "math", 
                "mmlu", 
                "programming"
]
DATASET_PATHS = [f"datasets/{name}_processed.csv" for name in DATASET_NAMES]
# Load and concatenate all datasets
df = pd.concat([pd.read_csv(path, encoding="utf-8-sig") for path in DATASET_PATHS], ignore_index=True)

In [7]:
# ! df.dropna(inplace=True)    # for analysis and vis keep all  
df.head()

,type,prompt,category,gemma-scope-2b-pt-res-canonical-layer_0/width_16k/canonical-top_mean_ids,gemma-scope-2b-pt-res-canonical-layer_0/width_16k/canonical-top_mean_vals,gemma-scope-2b-pt-res-canonical-layer_0/width_16k/canonical-token_feature_ids,gemma-scope-2b-pt-res-canonical-layer_1/width_16k/canonical-top_mean_ids,gemma-scope-2b-pt-res-canonical-layer_1/width_16k/canonical-top_mean_vals,gemma-scope-2b-pt-res-canonical-layer_1/width_16k/canonical-token_feature_ids,gemma-scope-2b-pt-res-canonical-layer_2/width_16k/canonical-top_mean_ids,gemma-scope-2b-pt-res-canonical-layer_2/width_16k/canonical-top_mean_vals,gemma-scope-2b-pt-res-canonical-layer_2/width_16k/canonical-token_feature_ids,gemma-scope-2b-pt-res-canonical-layer_3/width_16k/canonical-top_mean_ids,gemma-scope-2b-pt-res-canonical-layer_3/width_16k/canonical-top_mean_vals,gemma-scope-2b-pt-res-canonical-layer_3/width_16k/canonical-token_feature_ids,gemma-scope-2b-pt-res-canonical-layer_4/width_16k/canonical-top_mean_ids,gemma-scope-2b-pt-res-canonical-layer_4/width_16k/canonical-top_mean_vals,gemma-scope-2b-pt-res-canonical-layer_4/width_16k/canonical-token_feature_ids,gemma-scope-2b-pt-res-canonical-layer_5/width_16k/canonical-top_mean_ids,gemma-scope-2b-pt-res-canonical-layer_5/width_16k/canonical-top_mean_vals,gemma-scope-2b-pt-res-canonical-layer_5/width_16k/canonical-token_feature_ids,gemma-scope-2b-pt-res-canonical-layer_6/width_16k/canonical-top_mean_ids,gemma-scope-2b-pt-res-canonical-layer_6/width_16k/canonical-top_mean_vals,gemma-scope-2b-pt-res-canonical-layer_6/width_16k/canonical-token_feature_ids,gemma-scope-2b-pt-res-canonical-layer_7/width_16k/canonical-top_mean_ids,gemma-scope-2b-pt-res-canonical-layer_7/width_16k/canonical-top_mean_vals,gemma-scope-2b-pt-res-canonical-layer_7/width_16k/canonical-token_feature_ids,gemma-scope-2b-pt-res-canonical-layer_8/width_16k/canonical-top_mean_ids,gemma-scope-2b-pt-res-canonical-layer_8/width_16k/canonical-top_mean_vals,gemma-scope-2b-pt-res-canonical-layer_8/width_16k/canonical-token_feature_ids,gemma-scope-2b-pt-res-canonical-layer_9/width_16k/canonical-top_mean_ids,gemma-scope-2b-pt-res-canonical-layer_9/width_16k/canonical-top_mean_vals,gemma-scope-2b-pt-res-canonical-layer_9/width_16k/canonical-token_feature_ids,gemma-scope-2b-pt-res-canonical-layer_10/width_16k/canonical-top_mean_ids,gemma-scope-2b-pt-res-canonical-layer_10/width_16k/canonical-top_mean_vals,gemma-scope-2b-pt-res-canonical-layer_10/width_16k/canonical-token_feature_ids,gemma-scope-2b-pt-res-canonical-layer_11/width_16k/canonical-top_mean_ids,gemma-scope-2b-pt-res-canonical-layer_11/width_16k/canonical-top_mean_vals,gemma-scope-2b-pt-res-canonical-layer_11/width_16k/canonical-token_feature_ids,gemma-scope-2b-pt-res-canonical-layer_12/width_16k/canonical-top_mean_ids,gemma-scope-2b-pt-res-canonical-layer_12/width_16k/canonical-top_mean_vals,gemma-scope-2b-pt-res-canonical-layer_12/width_16k/canonical-token_feature_ids,gemma-scope-2b-pt-res-canonical-layer_13/width_16k/canonical-top_mean_ids,gemma-scope-2b-pt-res-canonical-layer_13/width_16k/canonical-top_mean_vals,gemma-scope-2b-pt-res-canonical-layer_13/width_16k/canonical-token_feature_ids,gemma-scope-2b-pt-res-canonical-layer_14/width_16k/canonical-top_mean_ids,gemma-scope-2b-pt-res-canonical-layer_14/width_16k/canonical-top_mean_vals,gemma-scope-2b-pt-res-canonical-layer_14/width_16k/canonical-token_feature_ids,gemma-scope-2b-pt-res-canonical-layer_15/width_16k/canonical-top_mean_ids,gemma-scope-2b-pt-res-canonical-layer_15/width_16k/canonical-top_mean_vals,gemma-scope-2b-pt-res-canonical-layer_15/width_16k/canonical-token_feature_ids,gemma-scope-2b-pt-res-canonical-layer_16/width_16k/canonical-top_mean_ids,gemma-scope-2b-pt-res-canonical-layer_16/width_16k/canonical-top_mean_vals,gemma-scope-2b-pt-res-canonical-layer_16/width_16k/canonical-token_feature_ids,gemma-scope-2b-pt-res-canonical-layer_17/width_16k/canonical-top_mean_ids,gemma-scope-2b-pt-res-canonical-layer_17/width_16k/canonic

In [8]:
counts = df['type'].value_counts()
print(counts)

type
programming            5000
math                   4998
empathetic_dialogue    4992
mmlu                   4984
Name: count, dtype: int64


In [9]:
# df.dropna(inplace=True)
df.describe()

,gemma-scope-2b-pt-res-canonical-layer19-top_mean_ids,gemma-scope-2b-pt-res-canonical-layer19-top_mean_vals,gemma-scope-2b-pt-res-canonical-layer20-top_mean_ids,gemma-scope-2b-pt-res-canonical-layer20-top_mean_vals,gemma-scope-2b-pt-res-canonical-layer21-top_mean_ids,gemma-scope-2b-pt-res-canonical-layer21-top_mean_vals,gemma-scope-2b-pt-res-canonical-layer22-top_mean_ids,gemma-scope-2b-pt-res-canonical-layer22-top_mean_vals,gemma-scope-2b-pt-res-canonical-layer23-top_mean_ids,gemma-scope-2b-pt-res-canonical-layer23-top_mean_vals,gemma-scope-2b-pt-res-canonical-layer24-top_mean_ids,gemma-scope-2b-pt-res-canonical-layer24-top_mean_vals,gemma-scope-2b-pt-res-canonical-layer25-top_mean_ids,gemma-scope-2b-pt-res-canonical-layer25-top_mean_vals
count,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
df.count()

type                                                                              19974
prompt                                                                            19974
category                                                                          19974
gemma-scope-2b-pt-res-canonical-layer_0/width_16k/canonical-top_mean_ids          14973
gemma-scope-2b-pt-res-canonical-layer_0/width_16k/canonical-top_mean_vals         14973
gemma-scope-2b-pt-res-canonical-layer_0/width_16k/canonical-token_feature_ids     19968
gemma-scope-2b-pt-res-canonical-layer_1/width_16k/canonical-top_mean_ids          14973
gemma-scope-2b-pt-res-canonical-layer_1/width_16k/canonical-top_mean_vals         14973
gemma-scope-2b-pt-res-canonical-layer_1/width_16k/canonical-token_feature_ids     19968
gemma-scope-2b-pt-res-canonical-layer_2/width_16k/canonical-top_mean_ids          14973
gemma-scope-2b-pt-res-canonical-layer_2/width_16k/canonical-top_mean_vals         14973
gemma-scope-2b-pt-res-canonical-

# Dot product

In [11]:
import numpy as np
import pandas as pd
import ast

def get_element_wise_product(typ, layer, class1, class2, vec1, vec2, top_n=10):
    """
    Identifies neurons that contribute most significantly to the dot product
    between two frequency vectors.

    Parameters:
    -----------
    typ : str
        A label for the type of comparison.
    layer : int or str
        Layer identifier.
    class1 : str
        Name of the first class.
    class2 : str
        Name of the second class.
    vec1, vec2 : list, np.array, or str
        The two frequency vectors (or their literal strings).
    top_n : int
        Number of top neurons to return.

    Returns:
    --------
    pd.DataFrame
        Columns = ['type', 'layer', 'class1', 'class2', 'Neuron_ID', 'Contribution']
        Sorted by descending Contribution, up to top_n rows.
    """
    # to numpy arrays
    v1 = np.array(vec1, dtype=float)
    v2 = np.array(vec2, dtype=float)

    # element-wise contributions
    contrib = v1 * v2

    # build full DataFrame
    df = pd.DataFrame({
        'type':       typ,
        'layer':      layer,
        'class1':     class1,
        'class2':     class2,
        'Neuron_ID':  np.arange(contrib.shape[0]),
        'Contribution': contrib
    })

    # keep only positive contributions
    df = df[df['Contribution'] > 0]

    # ensure all columns present even if empty
    return df.reindex(columns=[
        'type', 'layer', 'class1', 'class2', 'Neuron_ID', 'Contribution'
    ])


In [12]:
# DATASET_TYPES = {
#     "empathetic_dialogue": df.loc[df["type"] == "empathetic_dialogue", "category"].unique(),
#     "math":                df.loc[df["type"] == "math",                "category"].unique(),
#     "mmlu":                df.loc[df["type"] == "mmlu",                "category"].unique(),
#     "programming":         df.loc[df["type"] == "programming",         "category"].unique(),
# }


DATASET_TYPES = [
                "empathetic_dialogue", 
                "math", 
                "mmlu", 
                "programming"
                ]

print(DATASET_TYPES)

['empathetic_dialogue', 'math', 'mmlu', 'programming']


In [13]:
# create freq of vectors [0, 0, 0, .., 0]
VECTOR_SIZE_SAE = 16384
def to_vector(specialized_df, col_name, binary_vector=False):
    """
    input:
        specialized_df: pandas df 
        col_name: col name with its layer and type 
    output:
        vector of size 16500
            max neuron 16383
    """
    shared_neurons = [0] * VECTOR_SIZE_SAE
    for row_list_str in specialized_df[col_name]:      # TODO: optimize with counter
        # skip missing entries
        if pd.isna(row_list_str):
            continue
        neuron_list = ast.literal_eval(row_list_str)
        for neuron in neuron_list:
            if binary_vector:
                shared_neurons[neuron] = 1
            else:
                shared_neurons[neuron] += 1
            
    return shared_neurons
    
# to_vector(df, "gemma-scope-2b-pt-res-canonical-layer0-token_feature_ids")
# first_list = ast.literal_eval(df["gemma-scope-2b-pt-res-canonical-layer0-token_feature_ids"].iloc[0] )  # now a Python list of ints

In [14]:
# DATASET_NAMES = ["emotion", "math", "mmlu", "programming"]
import itertools
import pandas as pd 
# for type/ layer  
# c1 
# c2 

TYPE = [
        "res", 
        "mlp", 
        "att",
       ]
LAYER = [i for i in range(26)]
results_list = []

for typ in TYPE:
    for layer in LAYER:
        # between class
        for class1, class2 in itertools.combinations(DATASET_TYPES, 2):
        # for class1 in DATASET_TYPES:
        #     for class2 in DATASET_TYPES:
        #         if class1 < class2:
        #             continue
            col = f"gemma-scope-2b-pt-{typ}-canonical-layer_{layer}/width_16k/canonical-token_feature_ids"
            
            # 1. get df 
            class1_filtered = df[df['type'] == class1]
            class2_filtered = df[df['type'] == class2]
            
            # 2. to vector
            vec_class1 = to_vector(class1_filtered, col, binary_vector=False)
            vec_class2 = to_vector(class2_filtered, col, binary_vector=False)
    
            print(class1, class2)

            # return df : type layer class1 class2 contrib
            # df += get_element_wise_product(typ, layer, class1, class2, vec_class1, vec_class2)

            # 3. Calculate the element-wise product and get the results as a DataFrame
            contribution_df = get_element_wise_product(typ, layer, class1, class2, vec_class1, vec_class2)
            
            # 4. Append the resulting DataFrame to our list
            if not contribution_df.empty:
                results_list.append(contribution_df)

empathetic_dialogue math
empathetic_dialogue mmlu
empathetic_dialogue programming
math mmlu
math programming
mmlu programming
empathetic_dialogue math
empathetic_dialogue mmlu
empathetic_dialogue programming
math mmlu
math programming
mmlu programming
empathetic_dialogue math
empathetic_dialogue mmlu
empathetic_dialogue programming
math mmlu
math programming
mmlu programming
empathetic_dialogue math
empathetic_dialogue mmlu
empathetic_dialogue programming
math mmlu
math programming
mmlu programming
empathetic_dialogue math
empathetic_dialogue mmlu
empathetic_dialogue programming
math mmlu
math programming
mmlu programming
empathetic_dialogue math
empathetic_dialogue mmlu
empathetic_dialogue programming
math mmlu
math programming
mmlu programming
empathetic_dialogue math
empathetic_dialogue mmlu
empathetic_dialogue programming
math mmlu
math programming
mmlu programming
empathetic_dialogue math
empathetic_dialogue mmlu
empathetic_dialogue programming
math mmlu
math programming
mmlu prog

In [15]:
# Concatenate all the individual result DataFrames into one large DataFrame
print("SAVE DATASET =======================================================")
final_df = pd.concat(results_list, ignore_index=True)

final_df = (
    final_df
    .sort_values('Contribution', ascending=False)
    .reset_index(drop=True)
)

# Save the final DataFrame to a CSV file
output_path = "./dot_product/neuron_contribution_between_class.csv"
final_df.to_csv(output_path, index=False)

print(f"Successfully saved all neuron contributions to '{output_path}'")
print("\nFinal DataFrame head:")
print(final_df.head())

SAVE DATASET =======================================================
Successfully saved all neuron contributions to './dot_product/neuron_contribution_between_class.csv'

Final DataFrame head:
  type  layer class1       class2  Neuron_ID  Contribution
0  mlp      7   math  programming       6117  8.144039e+11
1  mlp     24   math  programming       9329  7.782393e+11
2  mlp     23   math  programming      10184  7.565522e+11
3  res      1   math  programming      12054  6.470184e+11
4  mlp     24   math  programming        282  5.486137e+11
